In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# headers to get past captcha 
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
with requests.Session() as s:
   city = 'dallas/' 
   url = 'https://www.zillow.com/homes/for_sale/'+ city    
   r = s.get(url, headers=req_headers)

In [3]:
# Scrape pages (normally done with splinter but zillow blocks all bots)
i = 1
pages = {}

while i < 20:
    with requests.Session() as s:
        city = 'dallas/'
        url = f'https://www.zillow.com/homes/for_sale/{city}{i}_p'
        r = s.get(url, headers=req_headers)

        # save each page as a variable
        key = i
        soup = bs(r.content, 'html')
        pages[key] = soup
    
        # next page 
        i += 1

In [41]:
address_list = []
price_list = []
list_card = []

# loop through saved pages and scrape info
for i in range(1,20):
    soup = pages[i]
    
    listing = soup.find_all('ul', class_='photo-cards photo-cards_wow photo-cards_short')
    
    for lists in listing:
        address = lists.find_all('address', class_='list-card-addr')
        price = lists.find_all('div', class_='list-card-price')
        bd_ba_size = lists.find_all('ul', class_='list-card-details')
    
    for bd in bd_ba_size:
        list_card.append(bd.text)

    for a in address:
        address_list.append(a.text)

    for p in price:
        price_list.append(p.text)

In [42]:
# slice list card to extract beds, baths, size
# bds = list_card[0::3]
# baths = list_card[1::3]
# size = list_card[2::3]

In [43]:
# add to df 
house_info = pd.DataFrame({'address': address_list, 
                           'price': price_list,
                           'features': list_card})
house_info

,address,price,features
0,"2226 Pecan Grove Ct, Dallas, TX 75228","$503,900","3 bds3 ba2,434 sqft- House for sale"
1,"2827 Curvilinear Ct, Dallas, TX 75227","$245,000","4 bds2 ba2,200 sqft- New construction"
2,"9615 Limestone Dr, Dallas, TX 75217","$145,000","3 bds2 ba1,057 sqft- House for sale"
3,"2714 Burger Ave, Dallas, TX 75215","$155,000",3 bds2 ba912 sqft- House for sale
4,"351 Crenshaw Dr, Dallas, TX 75217","$199,000","4 bds2 ba1,360 sqft- House for sale"
...,...,...,...
755,"2937 Persimmon Rd, Dallas, TX 75241","$85,000",1.34 acres lot- Lot / Land for sale
756,"3458 Saint Cloud Cir, Dallas, TX 75229","$450,000","3 bds2 ba2,331 sqft- House for sale"
757,"5505 Charlestown Dr, Dallas, TX 75230","$1,495,000","5 bds5 ba4,897 sqft- House for sale"
758,"8435 Coppertowne Ln, Dallas, TX 75243","$325,000","3 bds2 ba1,855 sqft- Townhouse for sale"


## Clean data

In [44]:
# split address
address = house_info['address'].str.split(",", n = 2, expand = True)

# separate columns
house_info['address']= address[0]
house_info['city'] = address[1]
house_info['state'] = address[2]

# split zip code 
zipcode = house_info['state'].str.split(n=2, expand = True)
house_info['state'] = zipcode[0]
house_info['zipcode'] = zipcode[1]

In [45]:
# strip 'bds', 'ba', 'sqft'
beds = house_info['features'].str.split('bds', n = 2, expand = True)
baths = beds[1].str.split('ba', n = 2, expand = True)
size = baths[1].str.split(' ', n = 2, expand = True)
house_type = size[2].str.split(' ', n = 2, expand = True)

house_info['beds'] = beds[0]
house_info['baths'] = baths[0]
house_info['size'] = size[0]
house_info['type'] = house_type[0]
house_info

,address,price,features,city,state,zipcode,beds,baths,size,type
0,2226 Pecan Grove Ct,"$503,900","3 bds3 ba2,434 sqft- House for sale",Dallas,TX,75228,3,3,"2,434",House
1,2827 Curvilinear Ct,"$245,000","4 bds2 ba2,200 sqft- New construction",Dallas,TX,75227,4,2,"2,200",New
2,9615 Limestone Dr,"$145,000","3 bds2 ba1,057 sqft- House for sale",Dallas,TX,75217,3,2,"1,057",House
3,2714 Burger Ave,"$155,000",3 bds2 ba912 sqft- House for sale,Dallas,TX,75215,3,2,912,House
4,351 Crenshaw Dr,"$199,000","4 bds2 ba1,360 sqft- House for sale",Dallas,TX,75217,4,2,"1,360",House
...,...,...,...,...,...,...,...,...,...,...
755,2937 Persimmon Rd,"$85,000",1.34 acres lot- Lot / Land for sale,Dallas,TX,75241,1.34 acres lot- Lot / Land for sale,None,None,None
756,3458 Saint Cloud Cir,"$450,000","3 bds2 ba2,331 sqft- House for sale",Dallas,TX,75229,3,2,"2,331",House
757,5505 Charlestown Dr,"$1,495,000","5 bds5 ba4,897 sqft- House for sale",Dallas,TX,75230,5,5,"4,897",House
758,8435 Coppertowne Ln,"$325,000","3 bds2 ba1,855 sqft- Townhouse for sale",Dallas,TX,75243,3,2,"1,855",Townhouse


In [46]:
# Drop features column, null values, non-houses/apartments, repeat values
house_info = house_info.drop(columns=['features'])
house_info = house_info.dropna()
house_info = house_info[house_info.type != 'New']
house_info = house_info.replace('Home', 'House')
house_info = house_info.drop_duplicates(subset='address')

In [47]:
# Convert numbers to only numerical and strip price tag
house_info['price'] = house_info['price'].str.replace('$', '').str.replace(',', '')
house_info['size'] = house_info['size'].str.replace(',', '')

house_info

,address,price,city,state,zipcode,beds,baths,size,type
0,2226 Pecan Grove Ct,503900,Dallas,TX,75228,3,3,2434,House
2,9615 Limestone Dr,145000,Dallas,TX,75217,3,2,1057,House
3,2714 Burger Ave,155000,Dallas,TX,75215,3,2,912,House
4,351 Crenshaw Dr,199000,Dallas,TX,75217,4,2,1360,House
5,2212 Lowery St,265000,Dallas,TX,75215,3,2,1366,House
...,...,...,...,...,...,...,...,...,...
754,10226 Pinecrest Dr,249000,Dallas,TX,75228,2,1,988,House
756,3458 Saint Cloud Cir,450000,Dallas,TX,75229,3,2,2331,House
757,5505 Charlestown Dr,1495000,Dallas,TX,75230,5,5,4897,House
758,8435 Coppertowne Ln,325000,Dallas,TX,75243,3,2,1855,Townhouse


In [49]:
house_info.to_csv('./houses.csv')